In [ ]:
print("hello")

: 

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

notebooks_dir = Path.cwd()
project_dir = notebooks_dir.parent
data_dir = project_dir / 'data' / 'raw'
text_data_path = data_dir / 'Subtask_2_train.json'

: 

In [ ]:
text_data = json.loads(text_data_path.read_text())
for i in range(len(text_data[0]['conversation'])):
    print(text_data[0]['conversation'][i])

: 

In [ ]:
def print_format_conv(json_convo):
    for i in range(len(json_convo)):
        print(f"{json_convo[i]['speaker']} ({json_convo[i]['emotion']}): {json_convo[i]['text']}")

print_format_conv(text_data[0]['conversation'])

Chandler (neutral): Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .
All (neutral): Oh , yeah . Had that dream .
Chandler (surprise): Then I look down , and I realize there is a phone ... there .
Joey (surprise): Instead of ... ?
Chandler (anger): That is right .
Joey (neutral): Never had that dream .
Phoebe (neutral): No .
Chandler (neutral): All of a sudden , the phone starts to ring .


In [4]:
# extract conversation lines and put them into a single string with special sol and eol tokens
# and extract emotions to list 
def get_conversation_from_json(json_convo):
    convo = []
    emotions = []
    for line_idx in range(len(json_convo)):
        if line_idx == 0:
            convo += ['<soc> <sol> ' + json_convo[line_idx]['text'] + ' <eol> ']
        elif line_idx + 1 == len(json_convo):
            convo += ['<sol> ' + json_convo[line_idx]['text'] + ' <eol> <eoc>']
        else:
            convo += ['<sol> ' + json_convo[line_idx]['text'] + ' <eol> ']
        emotions += [json_convo[line_idx]['emotion']]
    return convo, emotions

convos = []
all_emotions = []
for convo_idx in range(len(text_data)):
    convo, emotions = get_conversation_from_json(text_data[convo_idx]['conversation'])
    convos += convo
    all_emotions += emotions

data = {'conversation' : convos, 'emotion': all_emotions}

In [5]:
df = pd.DataFrame.from_dict(data)
df.head()

,conversation,emotion
0,"<soc> <sol> Alright , so I am back in high sch...",neutral
1,"<sol> Oh , yeah . Had that dream . <eol>",neutral
2,"<sol> Then I look down , and I realize there i...",surprise
3,<sol> Instead of ... ? <eol>,surprise
4,<sol> That is right . <eol>,anger


In [6]:
#imports for text cleaning
import contractions
import re
import nltk
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from nltk.tokenize import wordpunct_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

#nltk.download('stopwords')
nltk.download('wordnet')

# helper function to convert the pos tag format into something compatible with the lemmatizer
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/teodorastereciu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# used to clean the text of a conversation
def clean_text(doc, lemmatizer):
    # expand contractions
    doc = contractions.fix(doc)
    
    # remove every char that is not alphanumeric, keep spaces
    doc = re.sub(r'\W+', ' ', doc)
    
    # tokenize into all lowercase tokens
    tokens = wordpunct_tokenize(doc) 
    tokens = [token.lower() for token in tokens]
    clean_tokens = ['<' + token + '>' if token in ['sol', 'eol', 'soc', 'eoc'] else token for token in tokens]
    
    # lemmatize tokens based on pos tags
    # aka get base form 
    '''pos = pos_tag(tokens)
    clean_tokens = []
    for item in pos:
        word = item[0]
        tag = item[1]
        if word in ['sol', 'eol']:
            clean_tokens.append('<' + word + '>')
        else:
            clean_tokens.append(lemmatizer.lemmatize(word, get_wordnet_pos(tag)))
    
    clean_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos]
    '''

    # add start of convo and end of convo markers
    #clean_conv_toks = ['<soc>'] + clean_tokens + ['<eoc>']
    return clean_tokens

wn_lemmatizer = WordNetLemmatizer()
df['clean_conversation'] = df['conversation'].apply(lambda convo: clean_text(convo, wn_lemmatizer))

In [14]:
print(df.head())

all_tokens = df['clean_conversation'].explode().tolist()
unique_count = len(set(all_tokens))
print(f"Vocabulary size (including special tokens): {unique_count}")

                                        conversation   emotion  \
0  <soc> <sol> Alright , so I am back in high sch...   neutral   
1          <sol> Oh , yeah . Had that dream . <eol>    neutral   
2  <sol> Then I look down , and I realize there i...  surprise   
3                      <sol> Instead of ... ? <eol>   surprise   
4                       <sol> That is right . <eol>      anger   

                                  clean_conversation  
0  [<soc>, <sol>, alright, so, i, am, back, in, h...  
1         [<sol>, oh, yeah, had, that, dream, <eol>]  
2  [<sol>, then, i, look, down, and, i, realize, ...  
3                        [<sol>, instead, of, <eol>]  
4                    [<sol>, that, is, right, <eol>]  
Vocabulary size (including special tokens): 6380


In [15]:
import tensorflow as tf

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

KeyboardInterrupt: 

: 

In [ ]:
max_features = 6000
sequence_length = 30

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [ ]:
train_text = df['clean_conversation'].tolist()
train_label = df['emotion'].tolist()

print(train_text)
print(train_label)

In [ ]:


vectorize_layer.adapt(train_text)

print(vectorize_layer.get_vocabulary())
print(len(vectorize_layer.get_vocabulary()))

# DRAFT

# Emotion recognition with a transformer-based encoder
Input is text of conversations

In [1]:
print("hello")

hello


In [3]:
import tensorflow as tf

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.9.2


In [4]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

notebooks_dir = Path.cwd()
project_dir = notebooks_dir.parent
data_dir = project_dir / 'data' / 'raw'
text_data_path = data_dir / 'Subtask_2_train.json'

In [5]:
text_data = json.loads(text_data_path.read_text())
for i in range(len(text_data[0]['conversation'])):
    print(text_data[0]['conversation'][i])

{'utterance_ID': 1, 'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .', 'speaker': 'Chandler', 'emotion': 'neutral', 'video_name': 'dia1utt1.mp4'}
{'utterance_ID': 2, 'text': 'Oh , yeah . Had that dream .', 'speaker': 'All', 'emotion': 'neutral', 'video_name': 'dia1utt2.mp4'}
{'utterance_ID': 3, 'text': 'Then I look down , and I realize there is a phone ... there .', 'speaker': 'Chandler', 'emotion': 'surprise', 'video_name': 'dia1utt3.mp4'}
{'utterance_ID': 4, 'text': 'Instead of ... ?', 'speaker': 'Joey', 'emotion': 'surprise', 'video_name': 'dia1utt4.mp4'}
{'utterance_ID': 5, 'text': 'That is right .', 'speaker': 'Chandler', 'emotion': 'anger', 'video_name': 'dia1utt5.mp4'}
{'utterance_ID': 6, 'text': 'Never had that dream .', 'speaker': 'Joey', 'emotion': 'neutral', 'video_name': 'dia1utt6.mp4'}
{'utterance_ID': 7, 'text': 'No .', 'speaker': 'Phoebe', 'emotion': 'neutral', 'video_name': 'dia1utt7.mp4'}

In [6]:
def print_format_conv(json_convo):
    for i in range(len(json_convo)):
        print(f"{json_convo[i]['speaker']} ({json_convo[i]['emotion']}): {json_convo[i]['text']}")

print_format_conv(text_data[0]['conversation'])

Chandler (neutral): Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .
All (neutral): Oh , yeah . Had that dream .
Chandler (surprise): Then I look down , and I realize there is a phone ... there .
Joey (surprise): Instead of ... ?
Chandler (anger): That is right .
Joey (neutral): Never had that dream .
Phoebe (neutral): No .
Chandler (neutral): All of a sudden , the phone starts to ring .


In [144]:
# extract conversation lines and put them into a single string with special sol and eol tokens
# and extract emotions to list 
def get_conversation_from_json(json_convo):
    convo = []
    emotions = []
    for line_idx in range(len(json_convo)):
        if line_idx == 0:
            convo += ['<soc> <sol> ' + json_convo[line_idx]['text'] + ' <eol> ']
        elif line_idx + 1 == len(json_convo):
            convo += ['<sol> ' + json_convo[line_idx]['text'] + ' <eol> <eoc>']
        else:
            convo += ['<sol> ' + json_convo[line_idx]['text'] + ' <eol> ']
        emotions += [json_convo[line_idx]['emotion']]
    return convo, emotions

convos = []
all_emotions = []
for convo_idx in range(len(text_data)):
    convo, emotions = get_conversation_from_json(text_data[convo_idx]['conversation'])
    convos += convo
    all_emotions += emotions

data = {'conversation' : convos, 'emotion': all_emotions}

In [147]:
print(len(data['conversation']))
print(len(data['emotion']))

13619
13619


In [ ]:
'''df = pd.DataFrame.from_dict(data)
df.head()

# imports for text cleaning
import contractions
import re
import nltk
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from nltk.tokenize import wordpunct_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

#nltk.download('stopwords')
nltk.download('wordnet')

# helper function to convert the pos tag format into something compatible with the lemmatizer
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# used to clean the text of a conversation
def clean_text(doc, lemmatizer):
    # expand contractions
    doc = contractions.fix(doc)
    
    # remove every char that is not alphanumeric, keep spaces
    doc = re.sub(r'\W+', ' ', doc)
    
    # tokenize into all lowercase tokens
    tokens = wordpunct_tokenize(doc) 
    tokens = [token.lower() for token in tokens]
    clean_tokens = ['<' + token + '>' if token in ['sol', 'eol'] else token for token in tokens]
    
    # lemmatize tokens based on pos tags
    # aka get base form 
    ''''''pos = pos_tag(tokens)
    clean_tokens = []
    for item in pos:
        word = item[0]
        tag = item[1]
        if word in ['sol', 'eol']:
            clean_tokens.append('<' + word + '>')
        else:
            clean_tokens.append(lemmatizer.lemmatize(word, get_wordnet_pos(tag)))
    #clean_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos]
    '''

    # add start of convo and end of convo markers
    '''clean_conv_toks = ['<soc>'] + clean_tokens + ['<eoc>']
    return clean_conv_toks

wn_lemmatizer = WordNetLemmatizer()
df['clean_conversation'] = df['conversation'].apply(lambda convo: clean_text(convo, wn_lemmatizer))'''

#df.head()

# all_tokens = df['clean_conversation'].explode().tolist()
# unique_count = len(set(all_tokens))
# print(f"Vocabulary size (including special tokens): {unique_count}")

In [21]:
import tensorflow as tf
from tensorflow.keras import layers

vocab_size = 6380
embedding_dim = 100
max_seq_length = 10

In [151]:
# convert to df dataset
dataset = tf.data.Dataset.from_tensor_slices((data["conversation"], data["emotion"]))

# Print a sample from the dataset
for conversation, emotions in dataset.take(3):
    print("Conversation line:", conversation.numpy().decode("utf-8"))
    print("Emotions:", emotions.numpy())

Conversation line: <soc> <sol> Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked . <eol> 
Emotions: b'neutral'
Conversation line: <sol> Oh , yeah . Had that dream . <eol> 
Emotions: b'neutral'
Conversation line: <sol> Then I look down , and I realize there is a phone ... there . <eol> 
Emotions: b'surprise'


In [174]:
train_examples = []
train_labels = []

for conversation_line, emotion in dataset:
    train_examples.append(conversation_line.numpy().decode("utf-8"))
    train_labels.append(emotion.numpy().decode("utf-8"))

print(train_examples[:10])
print(train_labels[:10])

['<soc> <sol> Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked . <eol> ', '<sol> Oh , yeah . Had that dream . <eol> ', '<sol> Then I look down , and I realize there is a phone ... there . <eol> ', '<sol> Instead of ... ? <eol> ', '<sol> That is right . <eol> ', '<sol> Never had that dream . <eol> ', '<sol> No . <eol> ', '<sol> All of a sudden , the phone starts to ring . <eol> <eoc>', '<soc> <sol> I do not want to be single , okay ? I just ... I just ... I just wanna be married again ! <eol> ', '<sol> And I just want a million dollars ! <eol> ']
['neutral', 'neutral', 'surprise', 'surprise', 'anger', 'neutral', 'neutral', 'neutral', 'sadness', 'neutral']


In [41]:
import contractions

def fix_contractions(input_data):
    return contractions.fix(input_data.numpy().decode("utf-8"))

def custom_standardization(input_data):
  fixed_contractions = tf.py_function(fix_contractions, [input_data], tf.string)
  lowercase = tf.strings.lower(fixed_contractions)
  # remove non-alphanumeric characters except '<' and '>'
  cleaned_text = tf.strings.regex_replace(lowercase, '[^a-zA-Z0-9<>]', ' ')
  # remove extra spaces
  cleaned_text = tf.strings.regex_replace(cleaned_text, ' +', ' ')
  return cleaned_text

In [42]:
for conversation, emotions in dataset.take(1):
    print("Original Conversation:", conversation.numpy().decode("utf-8"))
    print("Original Emotions:", emotions.numpy())

    # Apply custom standardization
    standardized_conversation = custom_standardization(conversation)
    print("Standardized Conversation:", standardized_conversation.numpy().decode("utf-8"))

Original Conversation: <soc> <sol> Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked . <eol> 
Original Emotions: b'neutral'
Standardized Conversation: <soc> <sol> alright so i am back in high school i am standing in the middle of the cafeteria and i realize i am totally naked <eol> 


In [94]:
max_features = 6000
sequence_length = 30

vectorize_layer = layers.TextVectorization(
    #max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [102]:
train_text = list(dataset.map(lambda x, y: x).as_numpy_iterator())
vectorize_layer.adapt(train_text)

print(vectorize_layer.get_vocabulary())
print(len(vectorize_layer.get_vocabulary()))

['', '[UNK]', 'sol', 'eol', 'i', 'you', 'is', 'the', 'not', 'it', 'to', 'a', 'that', 'and', 'do', 'oh', 'are', 'soc', 'what', 'eoc', 'am', 'no', 'know', 'have', 'we', 'me', 'this', 'so', 'my', 'okay', 'just', 'of', 'yeah', 'in', 'can', 'well', 'on', 'hey', 'was', 'all', 'for', 'with', 'right', 'he', 'be', 'but', 'will', 'your', 'here', 'like', 'go', 'there', 'did', 'get', 'out', 'gonna', 'would', 'about', 'uh', 'she', 'really', 'think', 'up', 'one', 'look', 'if', 'how', 'now', 'they', 'got', 'her', 'at', 'mean', 'see', 'him', 'ross', 'come', 'why', 'good', 'god', 'want', 'sorry', 'joey', 'going', 'could', 'who', 'tell', 'great', 'when', 'let', 'guys', 'hi', 'then', 'chandler', 'yes', 'time', 'were', 'umm', 'because', 'does', 'say', 'had', 'some', 'little', 'should', 'back', 'love', 'too', 'over', 'something', 'rachel', 'monica', 'guy', 'wait', 'been', 'doing', 'make', 'from', 'an', 'or', 'as', 'said', 'phoebe', 'where', 'them', 'ok', 'maybe', 'sure', 'wanna', 'never', 'ah', 'huh', 'thi

In [114]:
for text_batch, label_batch in dataset.take(1):
    print("Conversation lines:", text_batch)
    print("Emotion labels:", label_batch)

Conversation lines: tf.Tensor(b'<soc> <sol> Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked . <eol> ', shape=(), dtype=string)
Emotion labels: tf.Tensor(b'neutral', shape=(), dtype=string)


In [121]:
print("2 ---> ",vectorize_layer.get_vocabulary()[2])
print("3 ---> ",vectorize_layer.get_vocabulary()[3])
print("4 ---> ",vectorize_layer.get_vocabulary()[4])
print("5 ---> ",vectorize_layer.get_vocabulary()[5])
print("17 ---> ",vectorize_layer.get_vocabulary()[17])

2 --->  sol
3 --->  eol
4 --->  i
5 --->  you
17 --->  soc


In [119]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

for text_batch, label_batch in dataset.take(1):
    #text_batch, label_batch = text_batch[0], label_batch[0]
    print("Line", text_batch)
    print("Label", label_batch)
    print("Vectorized line", vectorize_text(text_batch, label_batch))


Line tf.Tensor(b'<soc> <sol> Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked . <eol> ', shape=(), dtype=string)
Label tf.Tensor(b'neutral', shape=(), dtype=string)
Vectorized line (<tf.Tensor: shape=(1, 30), dtype=int64, numpy=
array([[  17,    2,  326,   27,    4,   20,  105,   33,  345,  410,    4,
          20, 1088,   33,    7,  839,   31,    7, 2263,   13,    4,  654,
           4,   20,  236,  323,    3,    0,    0,    0]])>, <tf.Tensor: shape=(), dtype=string, numpy=b'neutral'>)


In [157]:
train_ds = dataset.map(vectorize_text)

In [135]:
embedding_dim = 16
model = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1, activation='sigmoid')])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          96000     
                                                                 
 dropout_2 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 96,017
Trainable params: 96,017
Non-trainable params: 0
__________________________________________________

In [138]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)])

In [170]:
# Example labels
labels = ['neutral', 'surprise', 'anger', 'sadness', 'joy', 'disgust', 'fear'] 
num_classes = 7

label_to_index = {label: index for index, label in enumerate(labels)}

# Function to encode a single example (line and emotion)
def encode_example(line, emotion):
    print(line)
    print(emotion)
    label = np.array(emotion).astype('str').decode("utf-8")
    index = label_to_index[label]
    one_hot = tf.one_hot(index, len(labels))
    return line, one_hot

# Map the encode_example function to the dataset
#encoded_dataset = dataset.map(lambda line, emotion: encode_example(line, emotion))

dataset.map(lambda line, emotion: print(line, emotion))

# Example usage
for line, label in dataset.take(5):
    print("Conversation line:", line)
    print(" label:", label)


for train in train_ds.take(1):
    print(train)

Tensor("args_0:0", shape=(), dtype=string) Tensor("args_1:0", shape=(), dtype=string)


InvalidArgumentError: Length for attr 'output_shapes' of 0 must be at least minimum 1
	; NodeDef: {{node MapDataset}}; Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=metadata:string,default=""> [Op:MapDataset]

In [152]:
epochs = 10
history = model.fit(
    train_ds.take(1000),
    epochs=epochs)

Epoch 1/10


ValueError: in user code:

    File "/Users/teodorastereciu/Documents/bachelors-project/mc-ecpe/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/teodorastereciu/Documents/bachelors-project/mc-ecpe/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/teodorastereciu/Documents/bachelors-project/mc-ecpe/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/teodorastereciu/Documents/bachelors-project/mc-ecpe/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/teodorastereciu/Documents/bachelors-project/mc-ecpe/.venv/lib/python3.10/site-packages/keras/engi

In [ ]:


class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, inputs):
        score = tf.nn.tanh(self.W(inputs))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128, return_sequences=True)),
    Attention(units=128),
    tf.keras.layers.Dense(num_emotions, activation='sigmoid') 
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=num_epochs, batch_size=batch_size)

loss, accuracy = model.evaluate(val_data, val_labels)

# Make predictions
predictions = model.predict(val_data)